In [ ]:
pip install pyspark==3.4.1 spark-nlp pandas matplotlib scikit_learn openpyxl

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, udf, lower, trim, regexp_replace, count, split
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import collect_list
from sklearn.metrics.pairwise import cosine_distances

import sparknlp
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, XlmRoBertaEmbeddings

# Bắt đầu tính giờ toàn bộ pipeline
total_start = time.time()

# Khởi tạo Spark NLP
spark = SparkSession.builder \
    .appName("Spark NLP Clustering") \
    .master("spark://172.18.0.2:7077") \
    .config("spark.driver.memory", "8g") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.0") \
    .getOrCreate()

sparknlp.start(spark)


# Đọc file JSONL
input_file_path = "/opt/workspace/gen_1604_formated.jsonl"
df = spark.read.option("multiLine", False).json(input_file_path)

# Trích xuất câu hỏi từ user
user_questions = df.select(explode("messages").alias("msg")) \
    .filter(col("msg.role") == "user") \
    .select(col("msg.content").alias("text")) \
    .filter(col("text").isNotNull())

# NLP pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
embeddings = XlmRoBertaEmbeddings.pretrained("xlm_roberta_base", "xx") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setCaseSensitive(False)
pipeline = Pipeline(stages=[document_assembler, tokenizer, embeddings])

# Chạy NLP pipeline
start = time.time()
model = pipeline.fit(user_questions)
embedded_data = model.transform(user_questions)
print(f"⏱ NLP Embedding Pipeline completed in {time.time() - start:.2f} seconds")

# UDF tính trung bình embedding
def avg_embeddings(embeddings):
    try:
        if embeddings and len(embeddings) > 0:
            vecs = [e['embeddings'] for e in embeddings if e['embeddings']]
            if vecs:
                avg = np.mean(vecs, axis=0)
                return Vectors.dense(avg.tolist())
    except:
        pass
    return None

avg_embeddings_udf = udf(avg_embeddings, VectorUDT())

# Vector hóa và chuẩn hóa
start = time.time()
vectorized_data = embedded_data.withColumn("features", avg_embeddings_udf(col("embeddings")))
vectorized_data = vectorized_data.filter(col("features").isNotNull())

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(vectorized_data)
scaled_data = scaler_model.transform(vectorized_data)
print(f"⏱ Feature scaling completed in {time.time() - start:.2f} seconds")

# KMeans clustering
start = time.time()
if scaled_data.count() >= 5:
    kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster", k=5)
    kmeans_model = kmeans.fit(scaled_data)
    clustered_data = kmeans_model.transform(scaled_data)
    print(f"⏱ KMeans clustering completed in {time.time() - start:.2f} seconds")
else:
    raise Exception("❗ Không đủ dữ liệu hợp lệ để phân cụm KMeans (yêu cầu >= 5).")

# Chuẩn hóa văn bản
normalized_data = clustered_data.withColumn(
    "normalized_text",
    trim(lower(regexp_replace(col("text"), "[\\p{Punct}]", "")))
)

# Đếm tần suất câu hỏi
start = time.time()
grouped = normalized_data.groupBy("normalized_text", "cluster") \
    .agg(count("*").alias("frequency")) \
    .orderBy(col("frequency").desc())
print("📋 Danh sách tất cả câu hỏi sau khi chuẩn hóa và phân cụm (theo tần suất):")
grouped.show(truncate=False, n=50)
print(f"⏱ Counting frequency completed in {time.time() - start:.2f} seconds")

# Vẽ biểu đồ tần suất theo cluster
cluster_counts = grouped.groupBy("cluster").sum("frequency") \
    .withColumnRenamed("sum(frequency)", "count") \
    .orderBy("cluster") \
    .toPandas()
plt.figure(figsize=(8, 5))
plt.bar(cluster_counts["cluster"], cluster_counts["count"], color="teal")
plt.xlabel("Cluster ID")
plt.ylabel("Number of Questions")
plt.title("Semantic Question Clustering Frequency")
plt.xticks(cluster_counts["cluster"])
plt.tight_layout()
plt.show()

# Tính medoid cho mỗi cụm
start = time.time()
clusters = clustered_data.select("cluster", "text", "features") \
    .groupBy("cluster") \
    .agg(
        collect_list("text").alias("questions"),
        collect_list("features").alias("features_list")
    ).collect()

def get_medoid_question(questions, features):
    vecs = np.array([v.toArray() for v in features])
    dists = cosine_distances(vecs)
    total_dists = dists.sum(axis=1)
    medoid_idx = np.argmin(total_dists)
    return questions[medoid_idx]

topic_table = []
for row in clusters:
    cluster_id = row["cluster"]
    questions = row["questions"]
    features = row["features_list"]
    topic = get_medoid_question(questions, features)
    frequency = len(questions)
    topic_table.append((cluster_id, topic, frequency))

topic_df = pd.DataFrame(topic_table, columns=["Cluster", "Topic", "Frequency"])
topic_df = topic_df.sort_values("Cluster")
print("\n📌 Chủ đề đại diện cho từng cụm:")
print(topic_df.to_string(index=False))
print(f"⏱ Topic medoid selection completed in {time.time() - start:.2f} seconds")

# Gán nhãn thủ công theo chủ đề
def classify_topic(text):
    text = text.lower()
    if any(word in text for word in ["báo hỏng", "sự cố", "bị hỏng", "trạng thái hỏng", "hỏng thiết bị"]):
        return 1
    elif any(word in text for word in ["bảo dưỡng", "lịch bảo dưỡng", "trạng thái bảo dưỡng"]):
        return 2
    elif any(word in text for word in ["điều chuyển", "chuyển thiết bị", "chuyển đến", "chuyển đi"]):
        return 3
    elif any(word in text for word in ["khu vực", "quản lý", "nhân sự", "chức vụ", "người quản lý", "thông tin cá nhân"]):
        return 4
    elif any(word in text for word in ["thiết bị", "tài sản", "loại thiết bị", "loại tài sản", "chứa tài sản"]):
        return 5
    else:
        return 0  # Không xác định

classify_topic_udf = udf(classify_topic)
final_df = normalized_data.withColumn("classified_topic", classify_topic_udf(col("normalized_text")))

# Thống kê theo nhãn phân loại chủ đề
final_stats = final_df.groupBy("classified_topic").count().orderBy("classified_topic")
print("\n📊 Thống kê số lượng câu hỏi theo chủ đề phân loại:")
final_stats.show()

# 🔽 Xuất ra file CSV và Excel
export_df = final_df.select("text", "normalized_text", "cluster", "classified_topic").toPandas()
export_df.to_csv("/opt/workspace/clustered_questions.csv", index=False, encoding='utf-8-sig')
export_df.to_excel("/opt/workspace/clustered_questions.xlsx", index=False)

print("\n✅ Đã xuất dữ liệu câu hỏi ra file:")
print("   📄 /opt/workspace/clustered_questions.csv")
print("   📄 /opt/workspace/clustered_questions.xlsx")

# Tổng thời gian
print(f"\n🚀 Tổng thời gian toàn bộ pipeline: {time.time() - total_start:.2f} seconds")
